In [1]:
import itertools
import lime_explainer as lime

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_names = ['customer_id', 'monetary_dnn', 'monetary_btyd', 'frequency_dnn',
                 'frequency_btyd', 'recency', 'T', 'time_between',
                 'avg_basket_value', 'avg_basket_size', 'cnt_returns',
                 'has_returned', 'frequency_btyd_clipped', 'monetary_btyd_clipped',
                 'target_monetary_clipped', 'target_monetary']

In [3]:
explainer = lime.Explainer()

In [4]:
explainer.load_data(
  gcs_path = 'gs://ml-clv-data/model_data.csv',
  features_to_use = [1,3,5,6,7,8,9,10,11],
  feature_names = feature_names,
  target_idx = 15,
  skip_first=True 
)

In [5]:
explainer.create_cmle_client(
  gcp_project = 'ml-clv',
  gcp_model = 'dnn_airflow',
  gcp_model_version = 'v1',
  output_func = lambda x : x['predicted_monetary'] 
)

In [12]:
record, df = explainer.explain_random_record(numeric_rows=[1,2,3,7,8])

Intercept 8112.557044310286
Prediction_local [-884.25840127]
Right: -187.04930114746094


In [8]:
print(record)

[1224.0900000000001, 43.0, 256.0, 274.0, 5.95, 28.47, 18.91, 0.0, 0.0]


In [9]:
df

,representation,weight
0,T <= 306.00,-10825.846271
1,29.00 < frequency_dnn <= 52.00,-9976.733982
2,cnt_returns <= 0.00,-7491.042877
3,1167.98 < monetary_dnn <= 2423.27,-2145.994288
4,14.75 < avg_basket_size <= 22.66,1544.865588


In [10]:
mod_record = ','.join(['2000'] + [str(e) for e in record[1:]])
print('Mod Record: {}\n\n'.format(mod_record))
_, mod_df = explainer.explain_record(mod_record)

Mod Record: 2000,43.0,256.0,274.0,5.95,28.47,18.91,0.0,0.0


Intercept 21754.256665876586
Prediction_local [4659.79214245]
Right: 2286.3837890625


In [11]:
mod_df

,representation,weight
0,cnt_returns <= 0.00,-18728.706895
1,recency <= 291.00,4236.427836
2,29.00 < frequency_dnn <= 52.00,-3740.671404
3,has_returned <= 0.00,3237.424785
4,T <= 306.00,-2098.938845
